In [1]:
import inspect
import ast
from tvm.script.ir_builder import relax as relax_builder, ir as I, IRBuilder as IB
def jit(target="cpu"):
    assert target in ["cpu", "gpu"]
    def inner(fn):
        return JIT(fn, target=target)
    return inner

class JIT:
    def __init__(self, fn, target="cpu"):
        self.fn = fn
        self.target = target
    
    def __call__(self, *args, **kwargs):
        fn_src = inspect.getsource(self.fn)
        fn_ast = ast.parse(fn_src)
        print(ast.dump(fn_ast, indent=4))
        code_generator = CodeGenerator(fn_ast, self.target)
        code_generator.code_gen()

class CodeGenerator(ast.NodeVisitor):
    def __init__(self, fn_ast, target):
        self.fn_ast = fn_ast
        self.target = target
        self.ib = IB() # 创建一个 IR builder
        self.ir_module = None
    
    def code_gen(self):
        with self.ib:
            self.visit(self.fn_ast) # with.ib scope 内生成的 IR 会被记录下来
        module = self.ib.get()  # the first step in build a module!
        print(module)


    def visit(self, node):
        print("Visit " + node.__class__.__name__)
        return super().visit(node)
    
    def visit_Module(self, node: ast.Module):
        if self.ir_module:
            raise AssertionError("We should have only one module!")
        self.ir_module = I.ir_module()
        with self.ir_module:
            super().generic_visit(node)
        
    
    def visit_FunctionDef(self, node: ast.FunctionDef):
        pass # 中断遍历了 ? NOTE：不遍历 child node 了
    
    def generic_visit(self, node: ast.AST):
        raise NotImplementedError("Unsupported AST node type: {}".format(type(node).__name__))


@jit(target="cpu")
def add():
    print("add")

add()

[12:57:13] /home/ningzhangcaltech/Github_Repo/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.0.0git with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[12:57:13] /home/ningzhangcaltech/Github_Repo/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.0.0git with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[12:57:13] /home/ningzhangcaltech/Github_Repo/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.0.0git with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`


Module(
    body=[
        FunctionDef(
            name='add',
            args=arguments(
                posonlyargs=[],
                args=[],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Constant(value='add')],
                        keywords=[]))],
            decorator_list=[
                Call(
                    func=Name(id='jit', ctx=Load()),
                    args=[],
                    keywords=[
                        keyword(
                            arg='target',
                            value=Constant(value='cpu'))])])],
    type_ignores=[])
Visit Module
Visit FunctionDef
# from tvm.script import ir as I

@I.ir_module
class Module:
    pass
